# Astra target landing site example

## Derivation of Nozzle lift equation (fixed ascent rate)

An estimate of the bounds of Nozzle lift is required for scipy's differential evolution algorithm. We'll use a primitive approach, since the method calculates (integrates) the ascent rate equation anyway, and a penalty method will be used for ascent rates reaching outside the desired boundaries.

Force balance:

\begin{equation}
(m_{bal} + m_{pay} + m_{gas}) \frac{d^2z}{dt^2} = B - W - D = 0\\
B = W + D\\
V(\rho_{air} - \rho_{gas}) g = (m_{bal} + m_{pay})g + \frac{1}{2} \rho_{air} \left(\frac{dz}{dt}\right)^2 C_D A
\end{equation}

Then use balloon volume to get a 3rd order polynomial in terms of $r$

\begin{equation}
\frac{4}{3} \pi (\rho_{air} - \rho_{gas}) g r^3 - \frac{1}{2} \rho_{air} \left(\frac{dz}{dt}\right)^2 C_D \pi r^2 - (m_{bal} + m_{pay})g = 0
\end{equation}

Solve for r, and extract real positive root (should be unique: otherwise, raise an error). Then use the relation of gas mass (eqn 3 in "High-Altitude Gas Balloon Trajectory Prediction: A Monte Carlo Model") to obtain the nozzle lift:

\begin{equation}
m_{gas} = \rho^0_{gas} V^0 = \rho^0_{gas} \frac{L_N/g + m_{bal}}{\rho^0_{air} - \rho^0_{gas}}\\
L_N/g = \frac{4\pi}{3} r^3 (\rho_{air}-\rho_{gas}) - m_{bal} \hspace{1em} (kg)
\end{equation}

This is found in ```astra.flight_tools.nozzleLiftFixedAscent```, and has been validated with [habhub](http://habhub.org/calc/), which gives nozzle lift values of the same magnitude, and similar value (+- 10%)

## Ground distance equation

The Haversine formula is used to obtain ground distance $\Delta$ of the landing site from its target (assuming a spherical Earth), as

\begin{equation}
    \Delta = 2r\dot{}\mathrm{arcsin}\left( \sqrt{\mathrm{sin^2} \left( \frac{\phi_2 - \phi_1}{2}\right) + \mathrm{cos}(\phi_1) \mathrm{cos}(\phi_2) \mathrm{sin}^2 \left(\frac{\lambda_2 - \lambda_1}{2}\right)  }\right),
\end{equation}

where $\phi, \lambda$ refer to latitude and longitude respectively, and $r$ is the Earth's average radius ($=6371km$)

## 2D Optimization: Time + Nozzle Lift

In [1]:
%load_ext autoreload
%autoreload
import astra
from astra.target_landing import targetFlight
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import interp1d
from astra.weather import forecastEnvironment
import astra
import os
%matplotlib notebook

In [2]:
targetLat = 51.077214
targetLon = -1.1866875398775423
targetElev = 137

# Use a previous forecast (allows offline development)
launch_datetime = datetime.today() + timedelta(days=1)
# simEnvironment = forecastEnvironment(launchSiteLat=50.903824,      # deg
#                                      launchSiteLon=-1.63697,     # deg
#                                      launchSiteElev=114.0,           # m
#                                      dateAndTime=launch_datetime,
#                                      forceNonHD=True,
#                                      debugging=True,
#                                      requestAll=False  # False has better memory overhead
#                                     )
# # Set up the example input data files (from 24/04/2017, Daytona Beach)
# fileDict = {}
# for paramName in GFS_Handler.weatherParameters.keys():
#     fileDict[paramName] = os.path.join(os.path.dirname(astra.__file__),
#         '../test/example_data',
#         'gfs_0p50_06z.ascii?{}[12:15][0:46][231:245][545:571]'.format(paramName))
# simEnvironment.loadFromNOAAFiles(fileDict)
# inputs = {'launchSiteForecasts': [simEnvironment]}

simulator = targetFlight(start_dateTime=launch_datetime,
                 targetLat=targetLat,
                 targetLon=targetLon,
                 targetElev=targetElev,
                 launchSites=[(50.903824, -1.63697, 114.0)],
                 balloonModel='TA100',
                 balloonGasType="Helium",
                 nozzleLift=1,
                 trainEquivSphereDiam=0.1,
                 inflationTemperature=0.0,
                 payloadTrainWeight=0.38,
                 windowDuration=72,
                 HD=False,
                 maxFlightTime=18000,
                 parachuteModel=None,
                 debugging=True,
                 log_to_file=False,
                 progress_to_file=False,
                 outputFile=os.path.join(''))

In [ ]:
import copy
bestProfile_bf, X, Y, distances = simulator.bruteForce()
results_bf = copy.deepcopy(simulator.results)

In [3]:
import time
stime = time.time()
simulator.environment = simulator.launchSiteForecasts[0]
etime = time.time()
print("Time taken to download forecast and create interpolators: {}".foormat(etime - stime))

# Plot weather as a function of time over the date range:
xs = np.linspace(0, simulator.windowDuration, 3001)
ys = np.linspace(0, np.max(simulator.a))

for i in range(len(xs))
Ps = [simulator.environment.getPressure(simulator.launchsSiteLat,
                                        simulator.launchSiteLon,
                                        simulator.launchSiteElev, x)
          for x in xs]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xs, Ps)
ax.set_ylabel("Pressure")
ax.set_xlabel("Time after start date (hours)")


DEBUG:astra.weather:Fetched time zone data about the launch site: UTC offset is 1.000000 hours
DEBUG:astra.weather:Using UTC time 20/06/17 10:56
INFO:astra.weather:Preparing to download weather data for parameters:
DEBUG:astra.weather:    Launch site Latitude: 50.903824
DEBUG:astra.weather:    Launch site Longitude: -1.63697
DEBUG:astra.weather:    Launch time: 2017-06-20 10:56:13.756621
DEBUG:astra.GFS:Attempting to download cycle data.


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_06z.ascii?tmpprs[9:34][0:46][263:301][0:719]
DEBUG:astra.GFS:GFS cycle not found.
DEBUG:astra.GFS:Moving to next cycle
DEBUG:astra.GFS:Attempting to download cycle data.


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_00z.ascii?tmpprs[11:36][0:46][263:301][0:719]
DEBUG:astra.GFS:Temperature data downloaded


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_00z.ascii?hgtprs[11:36][0:46][263:301][0:719]
DEBUG:astra.GFS:GFS cycle not found.
DEBUG:astra.GFS:Moving to next cycle
DEBUG:astra.GFS:Attempting to download cycle data.


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170618/gfs_0p50_18z.ascii?tmpprs[13:38][0:46][263:301][0:719]
DEBUG:astra.GFS:Temperature data downloaded


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170618/gfs_0p50_18z.ascii?hgtprs[13:38][0:46][263:301][0:719]
DEBUG:astra.GFS:Altitude data downloaded


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170618/gfs_0p50_18z.ascii?ugrdprs[13:38][0:46][263:301][0:719]
DEBUG:astra.GFS:GFS cycle not found.
DEBUG:astra.GFS:Moving to next cycle
DEBUG:astra.GFS:Attempting to download cycle data.


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170618/gfs_0p50_12z.ascii?tmpprs[15:40][0:46][263:301][0:719]
DEBUG:astra.GFS:Temperature data downloaded


DEBUG:astra.GFS:Requesting URL: http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170618/gfs_0p50_12z.ascii?hgtprs[15:40][0:46][263:301][0:719]
ERROR:astra.GFS:Error while connecting to the GFS server.
Traceback (most recent call last):
  File "/home/pchambers/git/astra/simulator/astra/GFS.py", line 370, in _NOAA_request
    HTTPresponse = urlopen(requestURL)
  File "/home/pchambers/anaconda3/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/home/pchambers/anaconda3/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/home/pchambers/anaconda3/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/home/pchambers/anaconda3/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/home/pchambers/anaconda3/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)


HTTPError: HTTP Error 400: Bad Request

In [ ]:
# Visualise the best result paths
simulator.plotPaths3D()

In [ ]:
%autoreload
import astra

# fig2, ax2 = simulator.plotLandingSites()
simulator.results = results_bf
simulator.bestProfile = bestProfile_bf
print(np.shape(X), np.shape(Y), np.shape(distances))
fig3, ax3 = simulator.plotLandingSiteDistanceContours(X, Y, distances.T, appendLabel=' Brute Force')


## Nelder Mead Simplex Algorithm

In [ ]:
%autoreload
import astra

In [ ]:
# Optimize with Scipy:
res_scipy = simulator.optimizeTargetLandingSite(x0=[0.5*simulator.windowDuration, 1.0], options={"disp": True})
bestProfile_scipy = simulator.bestProfile
results_scipy = simulator.results
Xs_scipy = simulator.Xs

In [ ]:
# Optimize with scipy differential evolution
simulator.bestProfile = bestProfile_scipy
simulator.results = results_scipy
simulator.Xs = Xs_scipy
simulator.plotLandingSiteDistances(fig3, ax3, marker='rx', bestMarker='r*', appendLabel=' Nelder-Mead')

In [ ]:
print(res_scipy)

## L-BFGS-B Algorithm

In [ ]:
from astra.flight_tools import nozzleLiftFixedAscent

# Optimize with Scipy:
start_timedelta = 0
end_timedelta = simulator.windowDuration

nozzleLiftLowerBound = nozzleLiftFixedAscent(simulator.minAscentRate,
                simulator._balloonWeight, simulator.payloadTrainWeight,
                simulator.environment.inflationTemperature,
                simulator.environment.getPressure(simulator.launchSiteLat,
                                                  simulator.launchSiteLon,
                                                  simulator.launchSiteElev,
                                                  simulator.start_dateTime),
                simulator._gasMolecularMass, simulator.excessPressureCoeff,
                CD=(0.225 + 0.425)/2.)
nozzleLiftUpperBound = nozzleLiftFixedAscent(simulator.maxAscentRate,
                simulator._balloonWeight, simulator.payloadTrainWeight,
                simulator.environment.inflationTemperature,
                simulator.environment.getPressure(simulator.launchSiteLat,
                                                  simulator.launchSiteLon,
                                                  simulator.launchSiteElev,
                                                  simulator.start_dateTime),
                simulator._gasMolecularMass, simulator.excessPressureCoeff,
                CD=(0.225 + 0.425)/2.)

res_lbfgsb = simulator.optimizeTargetLandingSite(x0=[0.5*simulator.windowDuration, 1.0],
                                                 method='L-BFGS-B',
                                                 bounds=([start_timedelta, end_timedelta],
                                                         [nozzleLiftLowerBound, nozzleLiftUpperBound]),
                                                 options={"disp": True, "maxfun":200},
                                                 )
bestProfile_lbfgsb = simulator.bestProfile
results_lbfgsb = simulator.results
Xs_lbfgsb = simulator.Xs

In [ ]:
# Optimize with scipy differential evolution
simulator.bestProfile = bestProfile_lbfgsb
simulator.results = results_lbfgsb
simulator.Xs = Xs_lbfgsb
simulator.plotLandingSiteDistances(fig3, ax3, marker='cx', bestMarker='c*', appendLabel=' L-BFGS-B')

In [ ]:
print(res_lbfgsb)

## Scipy differential Evolution

In [ ]:
%autoreload
import astra

res_de = simulator.optimizeTargetLandingSite(method='DE', tol=1, maxiter=6)
results_de = simulator.results
bestProfile_de = simulator.bestProfile
Xs_de = simulator.Xs

In [ ]:
res_de

In [ ]:
simulator.results = results_de
simulator.bestProfile = bestProfile_de
simulator.Xs = Xs_de
simulator.plotLandingSiteDistances(fig3, ax3, marker='kx', bestMarker='k*', appendLabel=' DE')

In [ ]:
fig3.savefig('Distance2D_NozzleLift_LaunchTime.pdf')

In [ ]:
Nx = simulator.windowDuration / 4 + 1
Ny = 21
print("Brute Force: delta={}, nevals={}".format(bestProfile_bf.distanceFromTarget, Nx * Ny))
print("Differential Evolution: delta={}, nevals={}".format(bestProfile_de.distanceFromTarget), res_de.nfev)
print("Nelder Mead: {}, nevals={}".format(bestProfile_scipy.distanceFromTarget, res_scipy.nfev))

## Testing

In [ ]:
 datetimeVector = [self.start_dateTime + timedelta(hours=t)
                            for t in range(self.windowDuration)]

nozzleLiftLowerBound = 0.4
nozzleLiftUpperBound = 1.0
nozzelLift_Vector = np.linspace(nozzleLiftLowerBound, nozzleLiftUpperBound)
self.results = {}

distance_map = {}
for i, launchSiteForecast in enumerate(self.launchSiteForecasts):
    self.environment = launchSiteForecast

    for j, t in enumerate(datetimeVector):
        distance_lift_vec = np.zeros(np.length(nozzelLift_Vector))
        for k, L in enumerate(nozzelLift_Vector):
            # brute force approach
            distance = self.targetDistance(t)
            distance_lift_vec[k] = distance

        distance_map[t] = distance_Vector  

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
x = np.linspace(-3, 3, 50)
y = np.exp(-x**2) + 0.1 * np.random.randn(50)
plt.plot(x, y, 'ro', ms=5)
# Use the default value for the smoothing parameter:

spl = UnivariateSpline(x, y)
xs = np.linspace(-3, 3, 1000)
plt.plot(xs, spl(xs), 'g', lw=3)
# Manually change the amount of smoothing:

spl.set_smoothing_factor(0.5)
plt.plot(xs, spl(xs), 'b', lw=3)

plt.show()

In [4]:
# This url appears to have problems in some circumstances - is it a cycle not available error, or something else?
url = "http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_06z.ascii?hgtprs[10:35][0:46][263:301][0:719]"

from six.moves.urllib.request import urlopen

for i in range(20):
    HTTPresponse = urlopen(url)
    response = HTTPresponse.read().decode('utf-8')
    if response[0] == "<":
        print(response)

<html>
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
  <title>GrADS Data Server - error</title>
</head>
 <body bgcolor="#ffffff">
<a href="http://nomads.ncep.noaa.gov:9090/dods">GrADS Data Server</a>
<br>
<h2>GrADS Data Server - error</h2>
<hr><br>
The server could not fulfill this request:<p><b>http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_06z.ascii?hgtprs[10:35][0:46][263:301][0:719]</b><p> because of the following error:<p>
<b>subset operation failed; process exceeded time limit of 100 sec</b><p>
Check the syntax of your request, or click <a href=".help">here</a> for help using the server.
<br>
<hr><font size="-1"><a href="http://www.iges.org/grads/gds">
GrADS Data Server</a> 2.0 (<a href="http://nomads.ncep.noaa.gov:9090/dods/help">help&nbsp;using&nbsp;this&nbsp;server</a>)
. &nbsp;</font>
<br>
</body>
</html>



KeyboardInterrupt
Mon Jun 19 16:30:09 2017


KeyboardInterrupt: 